#### STA 141C - Big Data & High Performance Statistical Computing Spring 2022

#### Homework # 2

##### JONGWOOk CHOE

##### May 6, 2022

In [1]:
# library
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy.linalg import cho_factor, cho_solve
import matplotlib.pylab as plt
import scipy.sparse as sparse
import time
import math


1. Read in the ‘longley.dat’ with the response (number of people employed) in the first column and six explanatory variables in the other columns (GNP implicit price deflator, Gross National Product, number of unemployed, number of people in the armed forces, “noninstitutionalized” population % 14 years of age, year). Include an intercept in you model.

In [2]:
# Reading data using pandas
longley = pd.read_csv("longley.dat", header=None, delim_whitespace=True)

In [4]:
# Renaming the columns 
longley.rename(columns={0:"number of people employed", 1:"GNP implicit price deflator", 2: "Gross National Product", 3:"number of unemployed",
4: "number of people in the armed forces", 5: "noninstitutionalized population", 6: "year"}, inplace=True)

longley.head()

,number of people employed,GNP implicit price deflator,Gross National Product,number of unemployed,number of people in the armed forces,noninstitutionalized population,year
0,60323,83.0,234289,2356,1590,107608,1947
1,61122,88.5,259426,2325,1456,108632,1948
2,60171,88.2,258054,3682,1616,109773,1949
3,61187,89.5,284599,3351,1650,110929,1950
4,63221,96.2,328975,2099,3099,112075,1951


In [ ]:
# Linear Regression module
lm = LinearRegression()

# Y
Y = longley.iloc[:,0:1]

# X
X = longley.iloc[:,1:]

# fit data
model = lm.fit(X, Y)


print(model.coef_)
print(model.intercept_)


2. Assuming linear model $y \sim N(X\beta,\sigma^2I), \text{compute  } 1)$ regression coefficients $\hat{\beta} = (X^′X)^{−1}X^′y, 2)$ standard errors of $\hat{\beta} = \sigma^2\text{diag}((X^′X)^{−1}), \text{ and } 3)$ variance estimate $\hat{\sigma}^2 = (Y -X^′\hat{\beta})′(Y -X^′\hat{\beta})$ using following methods: GE/LU decomposition, Cholesky decomposition, and QR decomposition, and compare the computation speed for each method. Please compute them directly using numerical linear algebra functions; you can use the “black-box” function (e.g., lm() in R or sklearn.linear model.LinearRegression in python) only to check your results. (Hint: chol2inv() function in R computes the inverse of a matrix from its Cholesky factor. In python, you may try cho solve())

3. One popular regularization method is the ridge regression, which estimates regression coefficients by minimizing a penalized least squares criterion
$$\frac{1}{2}||y-X\beta||^2_2+\frac{\lambda}{2}||\beta||^2_2$$
show that the ridge solution is given by
$$\hat{\beta}_{\lambda}=(X'X +\lambda I_p)^{-1}X'y.$$

To show the ridge solution is given by 

$$\hat{\beta}_\lambda = (X^TX + \lambda I)^{-1}X^TY$$

we need to solve that

$$(Y - \beta^TX)^T(Y - \beta^TX) + \lambda\beta^T\beta$$ 

$$ => Y^TY - 2\beta^TX^TY + \beta^tX^TX\beta + \lambda\beta^T\beta  $$

First, we differentiate the above equation.

Then, we get


$$-2X^TY + 2X^TX\beta + 2\lambda\beta $$

Now, we have to solve it for 0 and then for $\hat{\beta}_\lambda$ 

$$-2X^TY + 2X^TX\beta + 2\lambda\beta = 0 $$

$$= -X^TY + X^TX\beta + \lambda\beta = 0 $$

$$= X^TX\beta + \lambda\beta = X^TY  $$

$$= (X^TX + \lambda I)\beta = X^TY $$

Hence, we have proved that

$$ \hat{\beta}_\lambda = (X^TX + \lambda I)^{-1}X^TY$$

4. Compute the ridge regression estimates $\hat{\beta}_{lambda}$ at a set of different values of $\lambda (\text{e.g. }, 0, 1, 2, \cdots , 100)$ by solving it as a least squares problem. Plot the $l_2$-norm of the ridge coefficients $||\hat{\beta}_{\lambda}||$ as a function of $\lambda$. You can use either QR or Cholesky method.

In [ ]:
y = longley[['number of people employed']]
x = longley.iloc[:,1:]     

# create intercept 
itcpt = np.repeat(1,16).reshape(-1,1)

x = np.append(x, itcpt, axis = 1)
x = x[:,[6,0,1,2,3,4,5]]


X = x.T.dot(x)
Y = x.T.dot(y)

# lambda from 1 to 100
ly = np.arange(1,100,0.1) 


df = np.arange(1,8).reshape(1,-1) 

# compute and add penalty
for i in ly:
    pty = i * np.eye(7)
    X = X + pty
    c, low = cho_factor(X)
    btas = cho_solve((c,low),Y).reshape(1,-1)
    df = np.vstack((df, btas))

# remove the first r.n given
# combine to df
df = pd.DataFrame(df[1:,:]) 

plt.figure(figsize = (8,6))
plt.plot(ly, df[0], label='b_0')
plt.plot(ly, df[1], label='b_1')
plt.plot(ly, df[2], label='b_2')
plt.plot(ly, df[3], label='b_3')
plt.plot(ly, df[4], label='b_4')
plt.plot(ly, df[5], label='b_5')
plt.plot(ly, df[6], label='b_6')


plt.legend()

plt.xlabel('Lambda')
plt.ylabel('Beta')


plt.show()

5. Implement your code using parallel computing.

6. (Bonus 1 point) Find out which method is the lm() function in R is using? And which algorithm is being used? Or find out which method is the linear regression function (there are multiple, but only need to choose one) in numpy/scipy is using? And which algorithm is being used?